In [1]:
# Import our dependencies 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

!pip install s3fs
import s3fs

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.6) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [4]:
# import our input dataset from AWS s3 bucket
df = pd.read_csv('s3://databootcampblueteam-bucket/final_split_age_df.csv')
df

,Unnamed: 0,res_county,age_group,sex,ethnicity,death_yn
0,2,COLUMBIA,0 to 64 years,Female,NaN,NaN
1,3,INDIAN RIVER,0 to 64 years,Female,Hispanic/Latino,No
2,4,GADSDEN,0 to 64 years,Female,NaN,No
3,6,TAYLOR,0 to 64 years,Female,NaN,NaN
4,7,HERNANDO,0 to 64 years,Female,Hispanic/Latino,No
...,...,...,...,...,...,...
913577,1697477,NaN,65+ years,Male,Non-Hispanic/Latino,No
913578,1697478,NaN,65+ years,Male,Non-Hispanic/Latino,NaN
913579,1697479,NaN,65+ years,Male,Non-Hispanic/Latino,No
913580,1697480,NaN,65+ years,Male,Non-Hispanic/Latino,No


In [7]:
df["ethnicity"].value_counts()

Non-Hispanic/Latino    496564
Hispanic/Latino        252895
Name: ethnicity, dtype: int64

In [8]:
df["death_yn"].value_counts()

No     796004
Yes     25429
Name: death_yn, dtype: int64

In [9]:
df["sex"].value_counts()

Female    482952
Male      419137
Name: sex, dtype: int64

In [12]:
df["age_group"].value_counts()

0 to 64 years    758003
65+ years        155579
Name: age_group, dtype: int64

In [13]:
df["res_county"].value_counts()

MIAMI-DADE      184250
BROWARD         162402
PALM BEACH       68724
PINELLAS         37064
HILLSBOROUGH     32997
POLK             29364
DUVAL            23370
PASCO            22738
ORANGE           19044
VOLUSIA          17700
MARION           17641
ALACHUA          17381
OSCEOLA          17349
MANATEE          17348
ST. LUCIE        17271
LEE              16651
ESCAMBIA         13864
LEON             12610
COLLIER          11880
SEMINOLE         11448
SARASOTA         11142
ST. JOHNS        10945
HERNANDO         10902
LAKE             10263
OKALOOSA          9897
CLAY              8774
MARTIN            7479
INDIAN RIVER      7345
BAY               7125
CHARLOTTE         6731
PUTNAM            5182
SUMTER            4582
FLAGLER           4415
BREVARD           4185
MONROE            4066
GADSDEN           3677
COLUMBIA          3651
WALTON            3344
HENDRY            3210
CITRUS            3056
HARDEE            2574
SANTA ROSA        2412
BRADFORD          2369
NASSAU     

In [11]:
df.isnull().sum()

Unnamed: 0         0
res_county     12725
age_group          0
sex            11493
ethnicity     164123
death_yn       92149
dtype: int64

In [17]:
clean_df = df.dropna(subset=["ethnicity", "death_yn", "res_county", "sex"])

clean_df

,Unnamed: 0,res_county,age_group,sex,ethnicity,death_yn
1,3,INDIAN RIVER,0 to 64 years,Female,Hispanic/Latino,No
4,7,HERNANDO,0 to 64 years,Female,Hispanic/Latino,No
5,8,HENDRY,0 to 64 years,Female,Hispanic/Latino,No
6,9,PUTNAM,0 to 64 years,Female,Hispanic/Latino,No
7,10,CHARLOTTE,0 to 64 years,Female,Hispanic/Latino,No
...,...,...,...,...,...,...
911095,1693902,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,Yes
911096,1693905,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,No
911097,1693911,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,No
911098,1693913,MIAMI-DADE,65+ years,Male,Non-Hispanic/Latino,Yes


In [20]:
# Generate our categorical variable list
attrition_cat = clean_df.dtypes[df.dtypes == "object"].index.tolist()
attrition_cat

['res_county', 'age_group', 'sex', 'ethnicity', 'death_yn']

In [21]:
# Check the number of unique values in each column
clean_df[attrition_cat].nunique()

res_county    52
age_group      2
sex            2
ethnicity      2
death_yn       2
dtype: int64

In [22]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(clean_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,res_county_ALACHUA,res_county_BAKER,res_county_BAY,res_county_BRADFORD,res_county_BREVARD,res_county_BROWARD,res_county_CHARLOTTE,res_county_CITRUS,res_county_CLAY,res_county_COLLIER,res_county_COLUMBIA,res_county_DESOTO,res_county_DUVAL,res_county_ESCAMBIA,res_county_FLAGLER,res_county_GADSDEN,res_county_HARDEE,res_county_HENDRY,res_county_HERNANDO,res_county_HIGHLANDS,res_county_HILLSBOROUGH,res_county_INDIAN RIVER,res_county_JACKSON,res_county_LAKE,res_county_LEE,res_county_LEON,res_county_LEVY,res_county_MANATEE,res_county_MARION,res_county_MARTIN,res_county_MIAMI-DADE,res_county_MONROE,res_county_NASSAU,res_county_OKALOOSA,res_county_ORANGE,res_county_OSCEOLA,res_county_PALM BEACH,res_county_PASCO,res_county_PINELLAS,res_county_POLK,res_county_PUTNAM,res_county_SANTA ROSA,res_county_SARASOTA,res_county_SEMINOLE,res_county_ST. JOHNS,res_county_ST. LUCIE,res_county_SUMTER,res_county_TAYLOR,res_county_VOLUSIA,res_county_WAKULLA,res_county_WALTON,res_county_WASHINGTON,age_group_0 to 64 years,age_group_65+ years,sex_Female,sex_Male,ethnicity_Hispanic/Latino,ethnicity_Non-Hispanic/Latino,death_yn_No,death_yn_Yes
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0


In [23]:
# Split our preprocessed data into our features and y target arrays want to predict
y = encode_df["death_yn_Yes"].values
X = encode_df.drop(["death_yn_Yes","death_yn_No"],1).values

# Split the preprocessed data into a training and testing dataset useing sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [24]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  6


# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation="relu", input_dim=number_input_features))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


In [26]:
# Check the structure of the Sequential model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 354       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
16066/16066 [==============================] - 22s 1ms/step - loss: 0.1112 - accuracy: 0.9610
Epoch 2/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0933 - accuracy: 0.9645
Epoch 3/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0920 - accuracy: 0.9645
Epoch 4/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0913 - accuracy: 0.9646
Epoch 5/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0911 - accuracy: 0.9646
Epoch 6/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0908 - accuracy: 0.9646
Epoch 7/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0908 - accuracy: 0.9646
Epoch 8/50
16066/16066 [==============================] - 22s 1ms/step - loss: 0.0908 - accuracy: 0.9646
Epoch 9/50
16066/16066 [==============================] - 21s 1ms/step - loss: 0.0908 - accuracy: 0.9646
Epoch 10/50
16066/16066 [==============================

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

5356/5356 - 4s - loss: 0.0899 - accuracy: 0.9650
Loss: 0.08991964906454086, Accuracy: 0.9649642705917358
